# 0. Imports

In [1]:
from llmstudio import LLM
import os
import json
import dotenv
import requests

# Load environment variables from .env file
dotenv.load_dotenv()

True

Running LLMstudio Engine on http://localhost:50001 Running LLMstudio Tracking on http://localhost:50002 



# 1. Raw Vertex Call

In [53]:
model = 'gemini-1.5-flash'
api_key = os.getenv("GOOGLE_API_KEY")
url = f"https://generativelanguage.googleapis.com/v1beta/models/{model}:streamGenerateContent?alt=sse"
headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": api_key,
}

In [9]:
calculator = [
      {'name': 'multiply',
       'description': 'Returns the product of two numbers.',
       'parameters': {'type_': 'OBJECT',
       'properties': {
         'a': {'type_': 'NUMBER'},
         'b': {'type_': 'NUMBER'} },
       'required': ['a', 'b']} }]

calculator_2 = """function_declarations {
  name: "multiply"
  description: "Returns the product of two numbers."
  parameters {
    type_: OBJECT
    properties {
      key: "b"
      value {
        type_: NUMBER
      }
    }
    properties {
      key: "a"
      value {
        type_: NUMBER
      }
    }
    required: "a"
    required: "b"
  }
}"""

In [10]:
message = 'What is 9 times 10?'
data = {
  "system_instruction": {
    "parts": {
      "text": ""
    }
  },
  "contents": [
    {
      "role": "user",
      "parts": [
        {
          "text": "How many legs does a spider have?"
        }
      ]
    }
  ]
}

In [11]:
response = requests.post(url, headers=headers, json=data, stream=False)

In [12]:
response.text

'data: {"candidates": [{"content": {"parts": [{"text": "A"}],"role": "model"},"finishReason": "STOP","index": 0}],"usageMetadata": {"promptTokenCount": 9,"candidatesTokenCount": 1,"totalTokenCount": 10}}\r\n\r\ndata: {"candidates": [{"content": {"parts": [{"text": " spider has **eight** legs. \\n"}],"role": "model"},"finishReason": "STOP","index": 0,"safetyRatings": [{"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT","probability": "NEGLIGIBLE"},{"category": "HARM_CATEGORY_HATE_SPEECH","probability": "NEGLIGIBLE"},{"category": "HARM_CATEGORY_HARASSMENT","probability": "NEGLIGIBLE"},{"category": "HARM_CATEGORY_DANGEROUS_CONTENT","probability": "NEGLIGIBLE"}]}],"usageMetadata": {"promptTokenCount": 9,"candidatesTokenCount": 8,"totalTokenCount": 17}}\r\n\r\n'

# 2. Vertex Functions Tests - Str to OAI to Vertex

In [2]:
def convert_openai_to_vertexai(input_data):
    # Check if the input is a simple string
    if isinstance(input_data, str):
        # Return a Vertex AI formatted message with a user message
        return {
            "system_instruction": {
                "parts": {
                    "text": ""  # Empty system instruction
                }
            },
            "contents": [
                {
                    "role": "user",
                    "parts": [{"text": input_data}]
                }
            ]
        }
    
    # Validate if input_data is a list and each element is a dictionary with the correct structure
    if not isinstance(input_data, list) or not all(isinstance(msg, dict) and 'role' in msg and 'content' in msg for msg in input_data):
        raise ValueError("Input must be a list of dictionaries, each containing 'role' and 'content' keys.")

    # Initialize the Vertex AI format if the input is not a simple string
    vertexai_format = {
        "system_instruction": {
            "parts": {
                "text": ""
            }
        },
        "contents": []
    }
    
    # Loop through the OpenAI formatted messages
    for message in input_data:
        if message["role"] == "system":
            # Set the system instruction
            vertexai_format["system_instruction"]["parts"]["text"] = message["content"]
        elif message["role"] in ["user", "assistant"]:
            # Convert roles: 'assistant' -> 'model'
            role = "model" if message["role"] == "assistant" else "user"
            # Append the message to the contents list in Vertex AI format
            vertexai_format["contents"].append({
                "role": role,
                "parts": [{"text": message["content"]}]
            })
        else:
            raise ValueError(f"Invalid role: {message['role']}. Expected 'system', 'user', or 'assistant'.")
    
    return vertexai_format

# Example usage with a string input
simple_input = "How many legs does a spider have?"
vertexai_format = convert_openai_to_vertexai(simple_input)
print(json.dumps(vertexai_format, indent=2))

# Example usage with a valid list of OpenAI messages
openai_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello"},
    {"role": "assistant", "content": "Hi there!"}
]
vertexai_format = convert_openai_to_vertexai(openai_messages)
print(json.dumps(vertexai_format, indent=2))

# Example usage with an invalid input to trigger an error
try:
    invalid_input = [
        {"role": "system", "text": "You are a helpful assistant."}  # 'text' should be 'content'
    ]
    vertexai_format = convert_openai_to_vertexai(invalid_input)
except ValueError as e:
    print(f"Error: {e}")


{
  "system_instruction": {
    "parts": {
      "text": ""
    }
  },
  "contents": [
    {
      "role": "user",
      "parts": [
        {
          "text": "How many legs does a spider have?"
        }
      ]
    }
  ]
}
{
  "system_instruction": {
    "parts": {
      "text": "You are a helpful assistant."
    }
  },
  "contents": [
    {
      "role": "user",
      "parts": [
        {
          "text": "Hello"
        }
      ]
    },
    {
      "role": "model",
      "parts": [
        {
          "text": "Hi there!"
        }
      ]
    }
  ]
}
Error: Input must be a list of dictionaries, each containing 'role' and 'content' keys.


# 3. Vertex Chat test

In [1]:
from llmstudio import LLM

In [2]:
llm = LLM('newvertex/gemini-1.5-pro-latest')

Running LLMstudio Engine on http://localhost:50001 Running LLMstudio Tracking on http://localhost:50002 

tools: None
message: {'system_instruction': {'parts': {'text': 'You are a helpful assistant'}}, 'contents': [{'role': 'user', 'parts': [{'text': 'Hello'}]}], 'tools': None, 'tool_config': {'function_calling_config': {'mode': 'AUTO'}}}
chunk: {'parts': [{'text': 'Hello'}], 'role': 'model'}
chunk: {'parts': [{'text': '! 👋  What can I do for you today? 😊 \n'}], 'role': 'model'}


### 3.1 String format

In [3]:
llm.chat('Hello')

ChatCompletion(id='0b661363-776e-48a3-a65e-6e0d08306ec8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! 👋  What can I do for you today? 😊 \n', role='assistant', function_call=None, tool_calls=None))], created=1723800713, model='gemini-1.5-pro-latest', object='chat.completion', system_fingerprint=None, usage=None, session_id=None, chat_input='Hello', chat_output='Hello! 👋  What can I do for you today? 😊 \n', context=[{'role': 'user', 'content': 'Hello'}], provider='newvertex', deployment='gemini-1.5-pro-latest', timestamp=1723800713.506577, parameters={'top_p': 1, 'top_k': 1, 'temperature': 1, 'max_output_tokens': 8192, 'frequency_penalty': 0, 'presence_penalty': 0}, metrics={'input_tokens': 1, 'output_tokens': 16, 'total_tokens': 17, 'cost_usd': 1.7149999999999997e-05, 'latency_s': 1.8147468566894531, 'time_to_first_token_s': 1.448904037475586, 'inter_token_latency_s': 0.11205267906188965, 'tokens_per_second': 2.204164170476

### 3.2 OpenAI format

In [4]:
message = [{"role": "system", "content": "You are Jamaican."},
           {"role": "assistant", "content": "How can i help you?"},
           {"role": "user", "content": "Hello, how are you?"}]

llm.chat(message)

ChatCompletion(id='7fa9938b-1ec8-4b7a-a89b-34c0bb779dd0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Aright man, mi deh yah a gwaan! How tings gwaan wid you?  🇯🇲 😄 \n\n(Translation: I'm doing well, how are you?) \n", role='assistant', function_call=None, tool_calls=None))], created=1723799492, model='gemini-1.5-pro-latest', object='chat.completion', system_fingerprint=None, usage=None, session_id=None, chat_input='Hello, how are you?', chat_output="Aright man, mi deh yah a gwaan! How tings gwaan wid you?  🇯🇲 😄 \n\n(Translation: I'm doing well, how are you?) \n", context=[{'role': 'system', 'content': 'You are Jamaican.'}, {'role': 'assistant', 'content': 'How can i help you?'}, {'role': 'user', 'content': 'Hello, how are you?'}], provider='newvertex', deployment='gemini-1.5-pro-latest', timestamp=1723799492.7148879, parameters={'top_p': 1, 'top_k': 1, 'temperature': 1, 'max_output_tokens': 8192, 'frequency_penalty': 0, 'presence

# 4. Vertex function Calling

In [1]:
functions = {
  "function_declarations": [
    {
      "name": "get_weather",
      "description": "Gets the weather for a certain location.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The location we are getting the weather for. For example: San Francisco"
          }
        },
        "required": [
          "location"
        ]
      }
    },
    {
      "name": "get_time",
      "description": "Gets the current time for a certain location.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The location we are getting the current time for. For example: New York"
          }
        },
        "required": [
          "location"  
        ]
      }
    }
  ]
}

In [2]:
from llmstudio import LLM

Running LLMstudio Engine on http://localhost:50001 

In [3]:
llm = LLM('newvertex/gemini-1.5-pro-latest')


Running LLMstudio Tracking on http://localhost:50002 
tools: {'function_declarations': [{'name': 'get_weather', 'description': 'Gets the weather for a certain location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location we are getting the weather for. For example: San Francisco'}}, 'required': ['location']}}, {'name': 'get_time', 'description': 'Gets the current time for a certain location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location we are getting the current time for. For example: New York'}}, 'required': ['location']}}]}
message: {'system_instruction': {'parts': {'text': 'You are a helpfull assistant'}}, 'contents': [{'role': 'model', 'parts': [{'text': 'How can i help you?'}]}, {'role': 'user', 'parts': [{'text': 'What time is it in San Francisco?'}]}], 'tools': {'function_declarations': [{'name': 'get_weather', 'description': 'Gets the weather for a certain

In [4]:
message = [{"role": "system", "content": "You are a helpfull assistant"},
           {"role": "assistant", "content": "How can i help you?"},
           {"role": "user", "content": "What time is it in San Francisco?"}]

# llm.chat('Hello', tools = functions)
llm.chat(message, tools = functions)

ChatCompletion(id='8968a00d-8d0b-431c-a8fb-c2545b43e97b', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_testid1234', function=Function(arguments="{'location': 'San Francisco'}", name='get_time'), type='function')]))], created=1723811876, model='gemini-1.5-pro-latest', object='chat.completion', system_fingerprint=None, usage=None, session_id=None, chat_input='What time is it in San Francisco?', chat_output="{'location': 'San Francisco'}", context=[{'role': 'system', 'content': 'You are a helpfull assistant'}, {'role': 'assistant', 'content': 'How can i help you?'}, {'role': 'user', 'content': 'What time is it in San Francisco?'}], provider='newvertex', deployment='gemini-1.5-pro-latest', timestamp=1723811876.5563881, parameters={'top_p': 1, 'top_k': 1, 'temperature': 1, 'max_output_tokens': 8192, 'frequency_penalty': 0, 'presence_pena

In [5]:
chunk = {'parts': [{'functionCall': {'name': 'get_time', 'args': {'location': 'San Francisco'}}}, {'functionCall': {'name': 'get_weather', 'args': {'location': 'San Francisco'}}}], 'role': 'model'}

In [8]:
from openai.types.chat.chat_completion_chunk import (
    Choice,
    ChoiceDelta,
    ChoiceDeltaFunctionCall,
    ChoiceDeltaToolCall,
    ChoiceDeltaToolCallFunction,
)

from openai.types.chat import ChatCompletionChunk

import uuid
import time

model = 'gemini-1.0-pro'

if 'functionCall' in chunk['parts'][0]:
    for index, functioncall in  enumerate(chunk['parts']):

        first_chunk = ChatCompletionChunk(
                            id=str(uuid.uuid4()),
                            choices=[
                                Choice(
                                    delta=ChoiceDelta(
                                        role="assistant",
                                        tool_calls=[
                                            ChoiceDeltaToolCall(
                                                index=index,
                                                id="call_" + "testid1234",
                                                function=ChoiceDeltaToolCallFunction(
                                                    name=functioncall['functionCall']['name'],
                                                    arguments="",
                                                    type="function",
                                                ),
                                            )
                                        ],
                                    ),
                                    finish_reason=None,
                                    index=index,
                                )
                            ],
                            created=int(time.time()),
                            model=model,
                            object="chat.completion.chunk",
                        ).model_dump()
        print(first_chunk)
        
        midle_chunk = ChatCompletionChunk(
                            id=str(uuid.uuid4()),
                            choices=[
                                Choice(
                                    delta=ChoiceDelta(
                                        tool_calls=[
                                            ChoiceDeltaToolCall(
                                                index=index,
                                                function=ChoiceDeltaToolCallFunction(
                                                    arguments=str(functioncall['functionCall']['args']),
                                                ),
                                            )
                                        ],
                                    ),
                                    finish_reason=None,
                                    index=index,
                                )
                            ],
                            created=int(time.time()),
                            model=model,
                            object="chat.completion.chunk",
                        ).model_dump()
        print(midle_chunk)
final_chunk = ChatCompletionChunk(
                            id=str(uuid.uuid4()),
                            choices=[
                                Choice(
                                    delta=ChoiceDelta(),
                                    finish_reason="tool_calls",
                                    index=0,
                                )
                            ],
                            created=int(time.time()),
                            model=model,
                            object="chat.completion.chunk",
                        ).model_dump()
print(final_chunk)
    

0
{'id': '8ca9c165-31dc-4852-a4d8-d2d104ab1f04', 'choices': [{'delta': {'content': None, 'function_call': None, 'role': 'assistant', 'tool_calls': [{'index': 0, 'id': 'call_testid1234', 'function': {'arguments': '', 'name': 'get_time', 'type': 'function'}, 'type': None}]}, 'finish_reason': None, 'index': 0, 'logprobs': None}], 'created': 1723801960, 'model': 'gemini-1.0-pro', 'object': 'chat.completion.chunk', 'system_fingerprint': None, 'usage': None}
{'id': 'f5df5475-e923-435a-a060-ca19e18cd26f', 'choices': [{'delta': {'content': None, 'function_call': None, 'role': None, 'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': "{'location': 'San Francisco'}", 'name': None}, 'type': None}]}, 'finish_reason': None, 'index': 0, 'logprobs': None}], 'created': 1723801960, 'model': 'gemini-1.0-pro', 'object': 'chat.completion.chunk', 'system_fingerprint': None, 'usage': None}
1
{'id': 'f6536fc7-dd83-4941-9a83-2946ef5ed827', 'choices': [{'delta': {'content': None, 'function_call':

# 4. Raw Vertex function calling

In [1]:
functions = {
  "function_declarations": [
    {
      "name": "get_weather",
      "description": "Gets the weather for a certain location.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The location we are getting the weather for. For example: San Francisco"
          }
        },
        "required": [
          "location"
        ]
      }
    },
    {
      "name": "get_time",
      "description": "Gets the current time for a certain location.",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The location we are getting the current time for. For example: New York"
          }
        },
        "required": [
          "location"  
        ]
      }
    }
  ]
}

In [2]:
functions_json = json.dumps(functions)
functions_obj = json.loads(functions_json)
type(functions_obj)


NameError: name 'json' is not defined

In [3]:
import json
import requests
import os


# Prepare the data for the API call
api_key = os.getenv('GOOGLE_API_KEY')  # Replace this with your actual Google API key
headers = {'Content-Type': 'application/json'}
data = {
    "system_instruction": {
        "parts": {
            "text": "You are a helpful assistant."
        }
    },
    "contents": {
        "role": "user",
        "parts": {
            "text": "Hello"
        }
    },
    "tools": functions_obj,  # Use the parsed object instead of the JSON string
    "tool_config": {
        "function_calling_config": {"mode": "AUTO"}
    },
}

# Make the API call
response = requests.post(f'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?key={api_key}',
                         headers=headers, json=data, stream=True)

# Print the response content
print(response.text)


NameError: name 'functions_obj' is not defined

# 5. OpenAI tool call

In [1]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "Get time of a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                },
                "required": ["location"]
            },
        }
    },
]

In [2]:
from openai import OpenAI
import os

In [3]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [4]:
message = [{"role": "system", "content": "You are a helpfull assistant"},
           {"role": "assistant", "content": "How can i help you?"},
           {"role": "user", "content": "What is the weather in San Francisco? And what is the time?"}]

In [5]:
response = client.chat.completions.create(
            model='gpt-4o',
            messages=message,
            tools=tools,
            stream=True
        )

In [42]:
response

In [43]:
for chunk in response:
    print(chunk)

ChatCompletionChunk(id='chatcmpl-9woLM1b1qGErhTbXA3UBQf2FhUAho', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1723803572, model='gpt-4o-2024-05-13', object='chat.completion.chunk', system_fingerprint='fp_3aa7262c27', usage=None)
ChatCompletionChunk(id='chatcmpl-9woLM1b1qGErhTbXA3UBQf2FhUAho', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, role=None, tool_calls=[ChoiceDeltaToolCall(index=0, id='call_ASsylP9CsVhBLyyXGwIyY2fH', function=ChoiceDeltaToolCallFunction(arguments='', name='get_current_weather'), type='function')]), finish_reason=None, index=0, logprobs=None)], created=1723803572, model='gpt-4o-2024-05-13', object='chat.completion.chunk', system_fingerprint='fp_3aa7262c27', usage=None)
ChatCompletionChunk(id='chatcmpl-9woLM1b1qGErhTbXA3UBQf2FhUAho', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, role=None, tool_calls=[ChoiceDe

# VertexAI Agent

In [1]:
from llmstudio import LLM

In [2]:
from langchain.tools import tool

@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def cancel_ticket(ticket_number: str):
    """Use this to cancel a ticket"""
    return "Ticket cancelled"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"

@tool
def make_complaint(complaint: str):
    """Use this to forward a complaint to the complaint department"""
    return "Complaint forwarded. We appreciate your feedback."

tools = [get_departure, cancel_ticket, buy_ticket, make_complaint]

Running LLMstudio Tracking on http://localhost:50002 
Running LLMstudio Engine on http://localhost:50001 


In [3]:
from langchain_community.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent, create_openai_tools_agent
from langchain import hub
from llmstudio.llm.langchain import ChatLLMstudio

prompt = hub.pull("hwchase17/openai-functions-agent")
model = ChatLLMstudio(model_id='vertex/gemini-1.5-flash', temperature=0)
# model = ChatLLMstudio(model_id='openai/gpt-4o', temperature=0)
model = ChatOpenAI()

print(prompt)

agent = create_openai_tools_agent(llm=model, prompt=prompt, tools=tools)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

agent_executor.invoke({"input": "Can you buy me a ticket to madrid? At what time does my train deppart?"})

/opt/anaconda3/envs/llmstudiodev/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} partial_variables={'chat_history': []} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], te

{'input': 'Can you buy me a ticket to madrid? At what time does my train deppart?',
 'output': 'I have bought a ticket for you to Madrid. Your train will depart at 12:00 AM.'}

In [4]:
from langchain_community.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent, create_openai_tools_agent
from langchain import hub
from llmstudio.llm.langchain import ChatLLMstudio

prompt = hub.pull("hwchase17/openai-functions-agent")
# model = ChatLLMstudio(model_id='newvertex/gemini-1.5-pro-latest', temperature=0)
# model = ChatLLMstudio(model_id='openai/gpt-4o', temperature=0)
model = ChatOpenAI()

print(prompt)

agent = create_openai_tools_agent(llm=model, prompt=prompt, tools=tools)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

agent_executor.invoke({"input": "Can you buy me a ticket to madrid? At what time does my train deppart?"})

input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} partial_variables={'chat_history': []} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], te

Exception: Not Found